In [2]:
import time
import json
import joblib
import serial
import numpy as np
import tensorflow as tf
from telnetlib import Telnet
from collections import deque, defaultdict
from fuzzy_logic import BlinkConfidenceSystem
from fuzzy_logic_att import MindStateControlSystem

# --- BLINK SETUP ---
with open('windowed_feature_cols.json') as f:
    blink_feature_cols = json.load(f)
with open('preprocessing_meta.json') as f:
    blink_meta = json.load(f)
blink_window_size = blink_meta['window_size']

# --- MIND STATE SETUP ---
with open("feature_cols.json") as f:
    mind_feature_cols = json.load(f)
bands = ['delta', 'theta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta', 'lowGamma', 'highGamma']
le_att_classes = np.load("le_att_classes.npy", allow_pickle=True)
le_rel_classes = np.load("le_rel_classes.npy", allow_pickle=True)

class SmartHomeEEGController:
    def __init__(self):
        # Serial and EEG
        self.esp32 = serial.Serial('COM4', 115200, timeout=1)
        self.tn = Telnet('localhost', 13854)
        self.tn.write(b'{"enableRawOutput": false,"format":"Json","enableBlinkDetection": true,"enableESense": true,"enableSpectra": true}\n')
        time.sleep(1)

        # Blink Detector
        self.blink_model = tf.keras.models.load_model(
            'best_eeg_cnn_bilstm_focal.h5',
            custom_objects={'loss': self._custom_focal_loss([0.3, 1.0, 0.7], 2)}
        )
        self.window_scaler = joblib.load('scaler_seq.pkl')
        self.feats_scaler = joblib.load('scaler_feats.pkl')
        self.blink_feature_cols = blink_feature_cols
        self.blink_window_size = blink_window_size
        self.blink_buf = np.zeros((self.blink_window_size, len(self.blink_feature_cols)), dtype=float)
        self.prediction_history = deque(maxlen=5)
        self.blink_strength_history = deque(maxlen=5)
        self.blink_threshold = 60
        self.min_blink_strength = 60
        self.double_blink_interval = 1.0
        self.pending_blink = None
        self.fuzzy = BlinkConfidenceSystem()

        # Mind State Controller
        self.mind_control = MindStateControlSystem()
        self.mind_feature_cols = mind_feature_cols
        self.mind_model = tf.keras.models.load_model("best_eeg_cnn_bilstm.h5")
        self.window_size = 20
        self.seq_buffer = deque(maxlen=self.window_size)
        self.rolling_size = 10
        self.band_buffers = {band: deque(maxlen=self.rolling_size) for band in bands}
        self.band_history = defaultdict(list)
        self.attention_history = deque(maxlen=15)
        self.meditation_history = deque(maxlen=15)
        self.last_control_update = 0
        self.control_update_interval = 1.0
        self.last_attention = 50
        self.last_meditation = 50
        self.pwm_min = 50
        self.pwm_max = 255
        self.last_pwm = 140
        self.pwm_step = 5

    def _custom_focal_loss(self, alpha, gamma):
        alpha = tf.constant(alpha, dtype=tf.float32)
        def loss(y_true, y_pred):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
            ce = -y_true * tf.math.log(y_pred)
            weight = alpha * tf.pow(1 - y_pred, gamma)
            loss = weight * ce
            return tf.reduce_sum(loss, axis=-1)
        return loss

    def run(self):
        print("=== Smart Home EEG Controller (Blink & Mind-State Fan) ===")
        print("Format: [State] Att:Val(Δ)[Level] | Med:Val(Δ)[Level] | Net:Val")
        try:
            while True:
                # Read EEG JSON
                line = self.tn.read_until(b'\r').decode('utf-8', errors='ignore').strip()
                if not line:
                    continue
                try:
                    data_dict = json.loads(line)
                except Exception:
                    continue

                # === Blink Logic ===
                self._process_blink(data_dict)

                # === Mind-State Fan Logic ===
                self._process_mind_state(data_dict)

        except KeyboardInterrupt:
            print("\nShutting down...")
        finally:
            self._send_fan_pwm(0)
            self.tn.close()
            self.esp32.close()

    # ---------- BLINK CONTROL LOGIC ----------

    def _process_blink(self, data_dict):
        feats = self._extract_blink_features(data_dict)
        self._roll_blink_buffer(feats)
        raw_blink_strength = data_dict.get("blinkStrength", 0)
        now = time.time()
        if raw_blink_strength < self.min_blink_strength:
            # Still update buffer for feature consistency
            return

        self.blink_strength_history.append(raw_blink_strength)
        smoothed_strength = np.mean(list(self.blink_strength_history)[-3:]) if self.blink_strength_history else raw_blink_strength

        # Blink detection & prediction
        if smoothed_strength > self.blink_threshold:
            window_scaled = self.window_scaler.transform(self.blink_buf).reshape(
                1, self.blink_window_size, len(self.blink_feature_cols))
            engineered_feats = self._compute_blink_engineered_features(self.blink_buf)
            feats_scaled = self.feats_scaler.transform([engineered_feats])
            cnn_proba = self.blink_model.predict([window_scaled, feats_scaled], verbose=0)[0]
            cnn_pred = int(np.argmax(cnn_proba))
            # Only class 1 or 2 ("no blink" is class 0)
            if cnn_pred == 0:
                nonzero_preds = [p for p in self.prediction_history if p in [1,2]]
                cnn_pred = nonzero_preds[-1] if nonzero_preds else 1
            self.prediction_history.append(cnn_pred)
            smoothed_pred = self._get_smoothed_prediction()
            fuzzy_conf = self.fuzzy.calculate_confidence(smoothed_strength)

            blink_record = {
                'time': now,
                'strength': smoothed_strength,
                'raw_pred': cnn_pred,
                'smoothed_pred': smoothed_pred,
                'fuzzy_conf': fuzzy_conf
            }

            if self.pending_blink and (now - self.pending_blink['time']) <= self.double_blink_interval:
                self._trigger_double_blink(self.pending_blink, blink_record)
                self.pending_blink = None
            else:
                if self.pending_blink:
                    self._trigger_single_blink(self.pending_blink)
                self.pending_blink = blink_record

        # Handle pending single blink
        if self.pending_blink:
            elapsed = time.time() - self.pending_blink['time']
            if elapsed > self.double_blink_interval:
                self._trigger_single_blink(self.pending_blink)
                self.pending_blink = None

    def _extract_blink_features(self, d):
        es, bp = d.get('eSense', {}), d.get('eegPower', {})
        return [
            es.get('attention', 50),
            es.get('meditation', 50),
            bp.get('delta', 0),
            bp.get('theta', 0),
            bp.get('lowAlpha', 0),
            bp.get('highAlpha', 0),
            bp.get('lowBeta', 0),
            bp.get('highBeta', 0),
            bp.get('lowGamma', 0),
            bp.get('highGamma', 0),
            d.get('blinkStrength', 0),
            time.time()
        ]

    def _roll_blink_buffer(self, feats):
        self.blink_buf = np.roll(self.blink_buf, -1, axis=0)
        self.blink_buf[-1] = feats

    def _compute_blink_engineered_features(self, window):
        blink_idx = self.blink_feature_cols.index('blinkStrength')
        time_idx = self.blink_feature_cols.index('time')
        blink_vals = window[:, blink_idx]
        feats = [
            np.mean(blink_vals), np.std(blink_vals), np.min(blink_vals), np.max(blink_vals),
            blink_vals[-1], np.ptp(blink_vals),
            np.mean(np.diff(blink_vals)), np.std(np.diff(blink_vals)),
            np.sum(blink_vals > self.blink_threshold),
            np.sum(blink_vals == 0),
            np.min(window[:, time_idx]), np.max(window[:, time_idx]), np.ptp(window[:, time_idx])
        ]
        for band in ['delta','theta','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','highGamma']:
            idx = self.blink_feature_cols.index(band)
            feats += [np.mean(window[:, idx]), np.std(window[:, idx])]
        blinks = blink_vals >= self.blink_threshold
        blink_times = window[:, time_idx][blinks]
        feats.append((blink_times[-1] - blink_times[-2]) if len(blink_times) > 1 else 0.0)
        return feats

    def _get_smoothed_prediction(self):
        if not self.prediction_history:
            return 1
        preds = [p if p in [1,2] else 1 for p in self.prediction_history]
        counts = np.bincount(preds+[0])
        return np.argmax(counts) if np.argmax(counts) != 0 else 1

    def _trigger_single_blink(self, blink):
        log_pred = 1 if blink['smoothed_pred'] in [1,2] else 1
        self._send_servo('DOOR', 90)
        print(f"[Single Blink] | strength={int(blink['strength'])} | model={log_pred} | fuzzy={blink['fuzzy_conf']:.2f} | Servo=DOOR:90°")

    def _trigger_double_blink(self, first, second):
        log_pred1 = 1 if first['smoothed_pred'] in [1,2] else 1
        log_pred2 = 2 if second['smoothed_pred'] in [1,2] else 2
        print(f"[Double Blink] | strengths=({int(first['strength'])},{int(second['strength'])}) | models=({log_pred1},{log_pred2}) | fuzzy=({first['fuzzy_conf']:.2f},{second['fuzzy_conf']:.2f}) | Servo=WINDOW:90°")
        self._send_servo('WINDOW', 90)

    def _send_servo(self, target, angle):
        self.esp32.write(f"ServoAngle:{target}:{angle}\n".encode())

    # ---------- MIND STATE CONTROL LOGIC ----------

    def _process_mind_state(self, data_dict):
        feat_row = self._extract_mind_features(data_dict)
        if feat_row is not None:
            self.seq_buffer.append(feat_row)

        now = time.time()
        if now - self.last_control_update >= self.control_update_interval:
            esense = data_dict.get('eSense', {})
            current_att = esense.get('attention', 50)
            current_med = esense.get('meditation', 50)
            self.attention_history.append(current_att)
            self.meditation_history.append(current_med)
            smooth_att = np.mean(self.attention_history)
            smooth_med = np.mean(self.meditation_history)
            delta_att = smooth_att - self.last_attention
            delta_med = smooth_med - self.last_meditation
            att_level = self._get_level(smooth_att)
            med_level = self._get_level(smooth_med)
            effects = self.mind_control.calculate_effects(smooth_att, smooth_med)

            att_color = "\033[92m" if delta_att >= 0 else "\033[91m"
            med_color = "\033[92m" if delta_med < 0 else "\033[91m"
            net_color = "\033[92m" if effects['net_effect'] >= 0 else "\033[91m"
            reset_color = "\033[0m"
            print(f"[Mind State] {att_color}Att:{smooth_att:.0f}(Δ{delta_att:+.1f})[{att_level}]{reset_color} | "
                  f"{med_color}Med:{smooth_med:.0f}(Δ{delta_med:+.1f})[{med_level}]{reset_color} | "
                  f"{net_color}Net:{effects['net_effect']:+.2f}{reset_color}")

            # Fan speed stepwise logic
            if (att_color == "\033[92m" and med_color == "\033[91m") or (att_color == "\033[92m" and med_color == "\033[92m"):
                print("\033[92m[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER\033[0m")
                self.last_pwm = min(self.pwm_max, self.last_pwm + self.pwm_step)
            elif (att_color == "\033[91m" and med_color == "\033[92m") or (att_color == "\033[91m" and med_color == "\033[91m"):
                print("\033[91m[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER\033[0m")
                self.last_pwm = max(self.pwm_min, self.last_pwm - self.pwm_step)
            else:
                print("\033[93m[Mixed State] Fan: MODERATE\033[0m")
                # No change
            self._send_fan_pwm(self.last_pwm)
            self.last_attention = smooth_att
            self.last_meditation = smooth_med

            # EEG model prediction output (as in original code)
            if len(self.seq_buffer) >= self.window_size:
                seq = np.array(self.seq_buffer)[-self.window_size:]
                seq = seq.reshape(1, self.window_size, len(self.mind_feature_cols))
                try:
                    att_pred_prob, rel_pred_prob = self.mind_model.predict(seq, verbose=0)
                    att_pred_class = np.argmax(att_pred_prob, axis=1)[0]
                    rel_pred_class = np.argmax(rel_pred_prob, axis=1)[0]
                    att_label = le_att_classes[att_pred_class]
                    rel_label = le_rel_classes[rel_pred_class]
                    print(f"\033[94m[EEG Model] Attention Prediction: {att_label}, Relaxation Prediction: {rel_label}\033[0m")
                except Exception as e:
                    print(f"EEG prediction error: {e}")

            self.last_control_update = now

    def _extract_mind_features(self, d):
        try:
            es, bp = d.get('eSense', {}), d.get('eegPower', {})
            band_vals = [bp.get(band, 0) for band in bands]
            for i, band in enumerate(bands):
                self.band_buffers[band].append(band_vals[i])
                self.band_history[band].append(band_vals[i])
            alpha_sum = band_vals[2] + band_vals[3]
            beta_sum = band_vals[4] + band_vals[5]
            theta_beta = band_vals[1] / (beta_sum + 1e-6)
            alpha_beta = alpha_sum / (beta_sum + 1e-6)
            lowHigh_alpha = band_vals[2] / (band_vals[3] + 1e-6)
            lowHigh_beta = band_vals[4] / (band_vals[5] + 1e-6)
            gamma_beta = (band_vals[6] + band_vals[7]) / (beta_sum + 1e-6)
            alpha_theta = alpha_sum / (band_vals[1] + 1e-6)
            beta_theta = beta_sum / (band_vals[1] + 1e-6)
            alpha_theta_beta = alpha_sum / (band_vals[1] + beta_sum + 1e-6)
            theta_alpha = band_vals[1] / (alpha_sum + 1e-6)
            gamma_alpha = (band_vals[6] + band_vals[7]) / (alpha_sum + 1e-6)
            ratio_vals = [
                theta_beta, alpha_beta, lowHigh_alpha, lowHigh_beta, gamma_beta,
                alpha_theta, beta_theta, alpha_theta_beta, theta_alpha, gamma_alpha
            ]
            rollings = []
            for band in bands:
                buf = self.band_buffers[band]
                arr = np.array(buf) if buf else np.zeros(1)
                rollings += [
                    np.mean(arr), np.std(arr), np.min(arr), np.max(arr)
                ]
            deltas_ = []
            for band in bands:
                hist = self.band_history[band]
                curr = hist[-1]
                prev = hist[-self.rolling_size] if len(hist) >= self.rolling_size else hist[0]
                deltas_.append(curr - prev)
            total_power = sum(band_vals) + 1e-6
            norms_ = [val / total_power for val in band_vals]
            feat_vec = (
                band_vals + ratio_vals + rollings + deltas_ + norms_
            )
            if len(feat_vec) != len(self.mind_feature_cols):
                print(f"Feature count mismatch: got {len(feat_vec)}, expected {len(self.mind_feature_cols)}")
                return None
            return feat_vec
        except Exception as e:
            print(f"Feature extraction error: {e}")
            return None

    def _get_level(self, value):
        if value < 40:
            return "LOW"
        elif 40 <= value < 70:
            return "MEDIUM"
        else:
            return "HIGH"

    def _send_fan_pwm(self, pwm):
        try:
            self.esp32.write(f'FAN:{pwm}\n'.encode())
            print(f"[DEBUG] Sent PWM: {pwm}")
        except Exception as e:
            print(f"Error sending PWM to ESP32: {e}")

# ==== RUN IT ====
if __name__ == "__main__":
    SmartHomeEEGController().run()


=== Smart Home EEG Controller (Blink & Mind-State Fan) ===
Format: [State] Att:Val(Δ)[Level] | Med:Val(Δ)[Level] | Net:Val
[Mind State] Att:26(Δ-24.0)[LOW] | Med:47(Δ-3.0)[MEDIUM] | Net:+0.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 135
[Mind State] Att:38(Δ+12.0)[LOW] | Med:48(Δ+1.5)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 140
[Mind State] Att:52(Δ+14.0)[MEDIUM] | Med:44(Δ-4.8)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 145
[Mind State] Att:58(Δ+5.8)[MEDIUM] | Med:43(Δ-0.9)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 150
[Mind State] Att:59(Δ+1.2)[MEDIUM] | Med:46(Δ+3.6)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 155
[Mind State] Att:56(Δ-3.0)[MEDIUM] | Med:50(Δ+3.9)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 150
[Mind State] Att:53(Δ-2.6)[MEDIUM] | Med:53(Δ+2.7)[MEDIUM] | Net:+1.00
[Focu

[Mind State] Att:69(Δ+0.0)[MEDIUM] | Med:50(Δ+0.0)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 190
[EEG Model] Attention Prediction: High, Relaxation Prediction: Medium
[Mind State] Att:69(Δ+0.4)[MEDIUM] | Med:52(Δ+1.1)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 195
[EEG Model] Attention Prediction: High, Relaxation Prediction: Medium
[Mind State] Att:69(Δ+0.4)[MEDIUM] | Med:54(Δ+2.0)[MEDIUM] | Net:+1.00
[Focus ↑, Relax ↓ or Both ↑] Fan: FASTER
[DEBUG] Sent PWM: 200
[EEG Model] Attention Prediction: High, Relaxation Prediction: Medium
[Mind State] Att:69(Δ-0.8)[MEDIUM] | Med:56(Δ+2.5)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 195
[EEG Model] Attention Prediction: High, Relaxation Prediction: Medium
[Mind State] Att:68(Δ-0.4)[MEDIUM] | Med:57(Δ+1.3)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 190
[EEG Model] Attention Prediction: High, Relaxation

[Mind State] Att:49(Δ-3.4)[MEDIUM] | Med:49(Δ+0.5)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 115
[EEG Model] Attention Prediction: Medium, Relaxation Prediction: Medium
[Mind State] Att:46(Δ-3.4)[MEDIUM] | Med:50(Δ+0.8)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 110
[EEG Model] Attention Prediction: Medium, Relaxation Prediction: Medium
[Mind State] Att:44(Δ-2.1)[MEDIUM] | Med:50(Δ-0.1)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 105
[EEG Model] Attention Prediction: Medium, Relaxation Prediction: Medium
[Mind State] Att:43(Δ-0.7)[MEDIUM] | Med:49(Δ-0.5)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 100
[EEG Model] Attention Prediction: Medium, Relaxation Prediction: Medium
[Mind State] Att:42(Δ-0.5)[MEDIUM] | Med:49(Δ-0.5)[MEDIUM] | Net:+1.00
[Focus ↓, Relax ↑ or Both ↓] Fan: SLOWER
[DEBUG] Sent PWM: 100
[EEG Model] Attention Prediction: Medium, 